In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class AirfoilLSTMRegressor(nn.Module):
    def __init__(self, point_dim=2, lstm_hidden_dim=128, num_lstm_layers=2, 
                 fc_hidden_dim=64, output_dim=3, other_features_dim=2, dropout_prob=0.2):
        super(AirfoilLSTMRegressor, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=point_dim,
            hidden_size=lstm_hidden_dim,
            num_layers=num_lstm_layers,
            batch_first=True,
            bidirectional=False
        )

        self.fc1 = nn.Linear(lstm_hidden_dim + other_features_dim, fc_hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(fc_hidden_dim, output_dim)

    def forward(self, sequence_data, sequence_lengths, other_features):
        packed_input = pack_padded_sequence(
            sequence_data, 
            sequence_lengths.cpu(),
            batch_first=True, 
            enforce_sorted=False
        )
        
        packed_lstm_out, (h_n, _) = self.lstm(packed_input)
        shape_representation = h_n[-1,:,:]
        combined_features = torch.cat((shape_representation, other_features), dim=1)
        x = self.fc1(combined_features)
        x = self.relu(x)
        x = self.dropout(x)
        predictions = self.fc2(x)
        
        return predictions


#Przykładowe użycie

In [ ]:

point_dim = 2                 # (x, y)
lstm_hidden_dim = 128
num_lstm_layers = 2
fc_hidden_dim = 64
output_dim = 3                # CL, CD, CM
other_features_dim = 2        # Re, AoA

model = AirfoilLSTMRegressor(point_dim, lstm_hidden_dim, num_lstm_layers, 
                             fc_hidden_dim, output_dim, other_features_dim)
